# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data = data.loc[data['country'].isin(['Belarus','Ukraine','Poland','Russian Federation'])]

In [4]:
data[data['artist']=='[unknown]'].count()
data['artist'].replace('[unknown]', np.nan,inplace=True)
data.dropna(inplace=True)

In [5]:
user_data = data.groupby(by=['user', 'sex', 'country'])['artist'].apply(lambda x: ','.join(x)).reset_index()

In [6]:
user_data.head(100)

,user,sex,country,artist
0,35,m,Ukraine,"radiohead,the kooks,coldplay,nine inch nails,h..."
1,39,m,Russian Federation,"britney spears,a-ha,joss stone,christina aguil..."
2,50,f,Russian Federation,"the ataris,yann tiersen,the smashing pumpkins,..."
3,64,m,Poland,"apparat,drowning pool,armin van buuren,alexiso..."
4,82,m,Russian Federation,"p.o.d.,chimaira,arch enemy,scar symmetry,slipk..."
...,...,...,...,...
95,873,f,Poland,"amy winehouse,amy macdonald,hey,the beatles,ka..."
96,878,f,Ukraine,"30 seconds to mars,dir en grey,mindless self i..."
97,896,f,Russian Federation,"him,simple plan,the 69 eyes,the killers,the bi..."
98,900,m,Poland,"electric light orchestra,eagles,sheryl crow,th..."


In [7]:
dummy_data = user_data['artist'].str.get_dummies(',')
dummy_data.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,...,weezer,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,zero 7,Édith piaf
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
fpg_data = fpgrowth(dummy_data, min_support=0.01, use_colnames=True)
fpg_data.sort_values('support', ascending=False)

,support,itemsets
155,0.129048,(metallica)
108,0.123568,(the prodigy)
58,0.122073,(red hot chili peppers)
0,0.114599,(radiohead)
1,0.112108,(coldplay)
...,...,...
664,0.010463,"(slipknot, 30 seconds to mars)"
658,0.010463,"(blink-182, sum 41)"
656,0.010463,"(joy division, red hot chili peppers)"
648,0.010463,"(muse, sigur rós)"


In [9]:
apriori_data = apriori(dummy_data, min_support=0.01, use_colnames=True)
apriori_data.sort_values('support', ascending=False)

,support,itemsets
308,0.129048,(metallica)
477,0.123568,(the prodigy)
392,0.122073,(red hot chili peppers)
385,0.114599,(radiohead)
101,0.112108,(coldplay)
...,...,...
742,0.010463,"(marilyn manson, deftones)"
646,0.010463,"(the doors, björk)"
554,0.010463,"(nirvana, ac/dc)"
732,0.010463,"(tool, coma)"


In [10]:
fpg_rules = association_rules(fpg_data, metric="confidence", min_threshold=0.5)
fpg_rules.sort_values('lift',ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
69,(above & beyond),"(armin van buuren, paul van dyk)",0.021923,0.016442,0.010962,0.500000,30.409091,0.010601,1.967115
67,"(armin van buuren, paul van dyk)",(above & beyond),0.016442,0.021923,0.010962,0.666667,30.409091,0.010601,2.934230
37,"(armin van buuren, atb)",(ferry corsten),0.017937,0.022422,0.010463,0.583333,26.016667,0.010061,2.346188
66,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
64,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879
63,(above & beyond),(ferry corsten),0.021923,0.022422,0.011958,0.545455,24.327273,0.011467,2.150673
38,"(ferry corsten, atb)",(armin van buuren),0.011958,0.038864,0.010463,0.875000,22.514423,0.009999,7.689088
68,"(above & beyond, paul van dyk)",(armin van buuren),0.012955,0.038864,0.010962,0.846154,21.772189,0.010458,6.247384
20,(the pussycat dolls),(lady gaga),0.019930,0.029895,0.012456,0.625000,20.906250,0.011861,2.586946
62,(above & beyond),(paul van dyk),0.021923,0.030892,0.012955,0.590909,19.128299,0.012277,2.368931


In [11]:
apriori_rules = association_rules(apriori_data, metric="confidence", min_threshold=0.5)
apriori_rules.sort_values('lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32,"(armin van buuren, paul van dyk)",(above & beyond),0.016442,0.021923,0.010962,0.666667,30.409091,0.010601,2.934230
34,(above & beyond),"(armin van buuren, paul van dyk)",0.021923,0.016442,0.010962,0.500000,30.409091,0.010601,1.967115
48,"(armin van buuren, atb)",(ferry corsten),0.017937,0.022422,0.010463,0.583333,26.016667,0.010061,2.346188
31,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
3,(above & beyond),(ferry corsten),0.021923,0.022422,0.011958,0.545455,24.327273,0.011467,2.150673
4,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879
49,"(ferry corsten, atb)",(armin van buuren),0.011958,0.038864,0.010463,0.875000,22.514423,0.009999,7.689088
33,"(above & beyond, paul van dyk)",(armin van buuren),0.012955,0.038864,0.010962,0.846154,21.772189,0.010458,6.247384
26,(the pussycat dolls),(lady gaga),0.019930,0.029895,0.012456,0.625000,20.906250,0.011861,2.586946
5,(above & beyond),(paul van dyk),0.021923,0.030892,0.012955,0.590909,19.128299,0.012277,2.368931


In [12]:
fpg_rules[fpg_rules[('support')] > 0.015]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022
32,(tricky),(massive attack),0.031888,0.080219,0.017439,0.546875,6.817255,0.014881,2.029861
33,(ferry corsten),(armin van buuren),0.022422,0.038864,0.015446,0.688889,17.725641,0.014575,3.089366
42,(iron maiden),(metallica),0.058794,0.129048,0.031888,0.542373,4.202866,0.024301,1.903191
48,(tiësto),(armin van buuren),0.030892,0.038864,0.015446,0.500000,12.865385,0.014245,1.922272
49,(paul van dyk),(armin van buuren),0.030892,0.038864,0.016442,0.532258,13.695409,0.015242,2.054843
54,(breaking benjamin),(linkin park),0.035874,0.107623,0.018435,0.513889,4.774884,0.014575,1.835746


In [22]:
apriori_rules[(apriori_rules['consequent support'] > 0.12)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
19,(iron maiden),(metallica),0.058794,0.129048,0.031888,0.542373,4.202866,0.024301,1.903191
28,(megadeth),(metallica),0.021425,0.129048,0.011460,0.534884,4.144833,0.008695,1.872546
61,"(koЯn, system of a down)",(metallica),0.025909,0.129048,0.012955,0.500000,3.874517,0.009611,1.741903
65,"(nirvana, pink floyd)",(metallica),0.021923,0.129048,0.010962,0.500000,3.874517,0.008132,1.741903
66,"(nirvana, the beatles)",(metallica),0.024415,0.129048,0.012456,0.510204,3.953589,0.009306,1.778193


In [ ]:
apriori_rules[(apriori_rules['confidence'] > 0.58) & (apriori_rules['lift'] > 26)]

In [20]:
fpg_rules[(fpg_rules['lift'] > 8) & (fpg_rules['support'] > 0.016)]
apriori_rules[(apriori_rules['lift'] > 8) & (apriori_rules['support'] > 0.016)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022
8,(paul van dyk),(armin van buuren),0.030892,0.038864,0.016442,0.532258,13.695409,0.015242,2.054843


In [21]:
apriori_rules[(apriori_rules['antecedent support'] > apriori_rules['consequent support'])]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879
34,(above & beyond),"(armin van buuren, paul van dyk)",0.021923,0.016442,0.010962,0.500000,30.409091,0.010601,1.967115


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022
8,(paul van dyk),(armin van buuren),0.030892,0.038864,0.016442,0.532258,13.695409,0.015242,2.054843
